In [121]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1579239597531_0001,pyspark,idle,Link,Link,✔


In [122]:
sc


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

In [123]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col, to_timestamp
from pyspark.sql.types import *
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [124]:
ss = SparkSession.builder.getOrCreate()
sc = SparkContext.getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [125]:
import time
time1 = time.time()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [126]:
dataschema = StructType([
    StructField('taxi_id', DoubleType(), True),
    StructField('trip_start_timestamp', StringType(), True),
    StructField('trip_end_timestamp', StringType(), True),
    StructField('trip_seconds', DoubleType(), True),
    StructField('trip_miles', DoubleType(), True),
    StructField('pickup_census_tract', StringType(), True),
    StructField('dropoff_census_tract', StringType(), True),
    StructField('pickup_community_area', DoubleType(), True),
    StructField('dropoff_community_area', DoubleType(), True),
    StructField('fare', DoubleType(), True),
    StructField('tips', DoubleType(), True),
    StructField('tolls', DoubleType(), True),
    StructField('extras', DoubleType(), True),
    StructField('trip_totals', DoubleType(), True),
    StructField('payment_type', StringType(), True),
    StructField('company', DoubleType(), True),
    StructField('pickup_latitude', DoubleType(), True),
    StructField('pickup_longitude', DoubleType(), True),
    StructField('dropoff_latitude', DoubleType(), True),
    StructField('dropoff_longitude', DoubleType(), True),   
])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [127]:
def toDoubleSafe(v):
    try:
        return float(v)
    except ValueError:
        return str(v)
def listchange(x):
    return [toDoubleSafe(v) for v in x]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [128]:
temp_list = []
for i in range(1,13):
    rdd = sc.textFile("s3://chicagotaxispark/data/chicago_taxi_trips_2016_{:02}.csv".format(i))
    header = rdd.first()
    rdd = rdd.filter(lambda x: x != header).map(lambda x: x.split(','))
    rdd = rdd.filter(lambda x: (x[9] != 0)&(x[0]!= '') & (x[3]!= '') & (x[4]!= '') & (x[7]!= '') & (x[8]!= '') & (x[9]!= '') & (x[-5]!= ''))
    rdd = rdd.map(lambda x : listchange(x))
    data_df = ss.createDataFrame(rdd, dataschema)
    data_df = data_df.dropna(subset=['fare', 'pickup_community_area', 'company',
            'dropoff_community_area', 'trip_seconds','pickup_census_tract','dropoff_census_tract'])
    data_df = data_df.drop('pickup_census_tract')
    data_df = data_df.drop('dropoff_census_tract')
    data_df = data_df.withColumn('trip_start_timestamp',to_timestamp(data_df.trip_start_timestamp,
                                'yyyy-MM-dd HH:mm:ss'))
    data_df = data_df.withColumn('trip_end_timestamp',to_timestamp(data_df.trip_end_timestamp,
                                'yyyy-MM-dd HH:mm:ss'))
    temp_list.append(data_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [129]:
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [130]:
chicago_ride_data = unionAll(temp_list[0], temp_list[1], temp_list[2], temp_list[3], temp_list[4], temp_list[5], temp_list[6]
                            , temp_list[7], temp_list[8], temp_list[9], temp_list[10], temp_list[11])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [131]:
from pyspark.sql.functions import date_format

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [132]:
chicago_ride_data = chicago_ride_data.withColumn('day_of_week', date_format('trip_start_timestamp', 'u'))\
                  .withColumn('hour_of_start', date_format('trip_start_timestamp', 'H'))\
                    .drop('trip_start_timestamp', 'trip_end_timestamp')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [133]:
from pyspark.ml.feature import StringIndexer
si = StringIndexer(inputCol = 'payment_type', outputCol = 'payment_type_num')
chicago_ride_data = si.fit(chicago_ride_data).transform(chicago_ride_data).drop('payment_type').withColumnRenamed('payment_type_num', 'payment_type')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [134]:
from pyspark.sql.types import DoubleType
chicago_ride_data = chicago_ride_data.withColumn("day_of_week", chicago_ride_data["day_of_week"].cast(DoubleType()))
chicago_ride_data = chicago_ride_data.withColumn("hour_of_start", chicago_ride_data["hour_of_start"].cast(DoubleType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [135]:
chicago_ride_data = chicago_ride_data.drop('tips', 'tolls','extras','pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude')
chicago_ride_data = chicago_ride_data.drop('trip_totals')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [136]:
# y variable: trip_seconds, trip_miles, trip_totals

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [137]:
y = ['fare', 'trip_seconds', 'trip_miles']
X = ['taxi_id', 'pickup_community_area', 'dropoff_community_area' ,'company', 'day_of_week', 'hour_of_start', 'payment_type']


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [138]:
# Merging the data with Vector Assembler.
from pyspark.ml.feature import VectorAssembler
va = VectorAssembler(outputCol= 'features',
                     inputCols=X)
data1 = va.transform(chicago_ride_data).select('fare', 'features').withColumnRenamed('fare', 'label')
data2 = va.transform(chicago_ride_data).select('trip_seconds', 'features').withColumnRenamed('trip_seconds', 'label')
data3 = va.transform(chicago_ride_data).select('trip_miles', 'features').withColumnRenamed('trip_miles', 'label')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [139]:
# Create Training and Test data.
data1 = data1.randomSplit([0.8, 0.2])
train1 = data1[0].cache()
test1 = data1[1].cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [140]:
data2 = data2.randomSplit([0.8, 0.2])
train2 = data2[0].cache()
test2 = data2[1].cache()
data3 = data3.randomSplit([0.8, 0.2])
train3 = data3[0].cache()
test3 = data3[1].cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [141]:
time.time() - time1

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

48.96849083900452

In [86]:
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import date_format
from pyspark.ml.feature import OneHotEncoderEstimator
from pyspark.ml.regression import LinearRegression

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [87]:
time1 = time.time()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'float' object has no attribute 'time'
Traceback (most recent call last):
AttributeError: 'float' object has no attribute 'time'



### Gradient-Boosted Trees

### Fare

In [88]:
gbt = GBTRegressor(maxIter=10)
modelgbt = gbt.fit(train1)
predictions = modelgbt.transform(test1)
# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(predictions)
print("Mean Absolute Error (MAE) on test data(fair) = %g" % mae)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-----+--------------------+
|       prediction|label|            features|
+-----------------+-----+--------------------+
|28.23579310116222|  0.0|[149.0,33.0,60.0,...|
|6.990289027554132|  0.0|[207.0,32.0,32.0,...|
|6.990289027554132|  0.0|[207.0,32.0,32.0,...|
|9.892398667595868|  0.0|[249.0,76.0,76.0,...|
|6.524452641533576|  0.0|[251.0,6.0,6.0,10...|
+-----------------+-----+--------------------+
only showing top 5 rows

Mean Absolute Error (MAE) on test data(fair) = 2.6239

### trip_seconds

In [89]:
modelgbt2 = gbt.fit(train2)
predictions2 = modelgbt2.transform(test2)
# Select example rows to display.
predictions2.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator2 = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="mae")
mae2 = evaluator2.evaluate(predictions2)
print("Mean Absolute Error (MAE) on test data(fair) = %g" % mae2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----+--------------------+
|        prediction|label|            features|
+------------------+-----+--------------------+
|  458.737906915881|  0.0|[22.0,32.0,32.0,1...|
| 727.1017271391306|  0.0|[22.0,35.0,35.0,1...|
| 629.2713049352964|  0.0|[29.0,28.0,28.0,1...|
|491.50940024309733|  0.0|[37.0,8.0,8.0,8.0...|
| 481.6064663015568|  0.0|[50.0,3.0,3.0,8.0...|
+------------------+-----+--------------------+
only showing top 5 rows

Mean Absolute Error (MAE) on test data(fair) = 278.937

### trip_miles

In [90]:
modelgbt3 = gbt.fit(train3)
predictions3 = modelgbt3.transform(test3)
# Select example rows to display.
predictions3.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator3 = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="mae")
mae3 = evaluator3.evaluate(predictions3)
print("Mean Absolute Error (MAE) on test data(fair) = %g" % mae3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----+--------------------+
|        prediction|label|            features|
+------------------+-----+--------------------+
|0.9474485606895491|  0.0|[3.0,7.0,7.0,107....|
|1.0232506302335787|  0.0|[3.0,8.0,8.0,107....|
|1.0232506302335787|  0.0|[3.0,8.0,8.0,107....|
|1.0232506302335787|  0.0|[3.0,8.0,8.0,107....|
|0.9867540777309797|  0.0|[3.0,24.0,22.0,10...|
+------------------+-----+--------------------+
only showing top 5 rows

Mean Absolute Error (MAE) on test data(fair) = 1.50041

### Random Forest

### Fare

In [91]:
rf = RandomForestRegressor()
rfModel = rf.fit(train1)
pred_rf = rfModel.transform(test1)
pred_rf.select("prediction", "label", "features").show(5)
ev = RegressionEvaluator(predictionCol='prediction', labelCol='label', metricName='mae')
ev.evaluate(pred_rf)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----+--------------------+
|        prediction|label|            features|
+------------------+-----+--------------------+
| 27.64246893636211|  0.0|[149.0,33.0,60.0,...|
| 8.494047485541527|  0.0|[207.0,32.0,32.0,...|
| 8.907883967202647|  0.0|[207.0,32.0,32.0,...|
|16.431380275259002|  0.0|[249.0,76.0,76.0,...|
| 9.490355774725746|  0.0|[251.0,6.0,6.0,10...|
+------------------+-----+--------------------+
only showing top 5 rows

3.642223169693648

### trip_seconds

In [92]:
rf = RandomForestRegressor()
rfModel2 = rf.fit(train2)
pred_rf2 = rfModel.transform(test2)
pred_rf2.select("prediction", "label", "features").show(5)
ev2 = RegressionEvaluator(predictionCol='prediction', labelCol='label', metricName='mae')
ev2.evaluate(pred_rf2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-----+--------------------+
|       prediction|label|            features|
+-----------------+-----+--------------------+
|8.404479711458942|  0.0|[22.0,32.0,32.0,1...|
|9.457460158573294|  0.0|[22.0,35.0,35.0,1...|
|8.528227057464123|  0.0|[29.0,28.0,28.0,1...|
|8.546430695933104|  0.0|[37.0,8.0,8.0,8.0...|
|9.629903140795813|  0.0|[50.0,3.0,3.0,8.0...|
+-----------------+-----+--------------------+
only showing top 5 rows

759.3936828015507

### trip_miles

In [93]:
rf = RandomForestRegressor()
rfModel3 = rf.fit(train3)
pred_rf3 = rfModel.transform(test3)
pred_rf3.select("prediction", "label", "features").show(5)

ev3 = RegressionEvaluator(predictionCol='prediction', labelCol='label', metricName='mae')
ev3.evaluate(pred_rf3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-----+--------------------+
|       prediction|label|            features|
+-----------------+-----+--------------------+
|8.868399724003144|  0.0|[3.0,7.0,7.0,107....|
|9.130953901740245|  0.0|[3.0,8.0,8.0,107....|
|  9.0207960957916|  0.0|[3.0,8.0,8.0,107....|
|9.278494859327953|  0.0|[3.0,8.0,8.0,107....|
|8.852178278092735|  0.0|[3.0,24.0,22.0,10...|
+-----------------+-----+--------------------+
only showing top 5 rows

10.466211949816676

### Generalized Linear Regression

### Fare

In [94]:
glr = GeneralizedLinearRegression(family="gaussian", maxIter=20, regParam=0.5)
modelglr = glr.fit(train1)
predictions = modelglr.transform(test1)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(predictions)
print("Mean Absolute Error (MAE) on test data(fair) = %g" % mae)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----+--------------------+
|        prediction|label|            features|
+------------------+-----+--------------------+
| 20.52098919829097|  0.0|[149.0,33.0,60.0,...|
|15.711162104558571|  0.0|[207.0,32.0,32.0,...|
|16.593659695205904|  0.0|[207.0,32.0,32.0,...|
|33.591210365400535|  0.0|[249.0,76.0,76.0,...|
| 3.875632525087175|  0.0|[251.0,6.0,6.0,10...|
+------------------+-----+--------------------+
only showing top 5 rows

Mean Absolute Error (MAE) on test data(fair) = 6.80653

### trip_seconds

In [95]:
glr = GeneralizedLinearRegression(family="gaussian", maxIter=20, regParam=0.5)
modelglr2 = glr.fit(train2)
predictions2 = modelglr2.transform(test2)

# Select example rows to display.
predictions2.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator2 = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="mae")
mae2 = evaluator2.evaluate(predictions2)
print("Mean Absolute Error (MAE) on test data = %g" % mae2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----+--------------------+
|        prediction|label|            features|
+------------------+-----+--------------------+
|  858.286881609986|  0.0|[22.0,32.0,32.0,1...|
| 959.9785072453885|  0.0|[22.0,35.0,35.0,1...|
| 838.4782123182163|  0.0|[29.0,28.0,28.0,1...|
|444.33982777023414|  0.0|[37.0,8.0,8.0,8.0...|
|324.18903743631233|  0.0|[50.0,3.0,3.0,8.0...|
+------------------+-----+--------------------+
only showing top 5 rows

Mean Absolute Error (MAE) on test data = 427.795

### trips_miles

In [96]:
glr = GeneralizedLinearRegression(family="gaussian", maxIter=20, regParam=0.5)
modelglr3 = glr.fit(train3)
predictions3 = modelglr3.transform(test3)

# Select example rows to display.
predictions3.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator3 = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="mae")
mae3 = evaluator3.evaluate(predictions3)
print("Mean Absolute Error (MAE) on test data = %g" % mae3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----+--------------------+
|        prediction|label|            features|
+------------------+-----+--------------------+
|0.6632709606964435|  0.0|[3.0,7.0,7.0,107....|
|1.2544022802614072|  0.0|[3.0,8.0,8.0,107....|
| 1.477504633225561|  0.0|[3.0,8.0,8.0,107....|
|1.3151682332182753|  0.0|[3.0,8.0,8.0,107....|
| 2.503937455651164|  0.0|[3.0,24.0,22.0,10...|
+------------------+-----+--------------------+
only showing top 5 rows

Mean Absolute Error (MAE) on test data = 2.52563

In [ ]:
time.time() - time1

## Linear Regression

In [97]:
data=ss.read.parquet('s3://chicagotaxispark/data/aggregated')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [98]:
time1 = time.time()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'float' object has no attribute 'time'
Traceback (most recent call last):
AttributeError: 'float' object has no attribute 'time'



In [99]:
chicago_ride_data = data.withColumn('day_of_week', date_format('trip_start_timestamp', 'u'))\
                  .withColumn('hour_of_start', date_format('trip_start_timestamp', 'H'))\
                    .drop('trip_start_timestamp', 'trip_end_timestamp')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [100]:
features = ['taxi_id','pickup_community_area','dropoff_community_area','payment_type','company','day_of_week','hour_of_start']
categorical_features = ['taxi_id','pickup_community_area','dropoff_community_area','payment_type','company','day_of_week','hour_of_start']
labels = ['fare','trip_seconds','trip_miles']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [101]:
si = StringIndexer(inputCol = 'payment_type', outputCol = 'payment_type_num')
sm = si.fit(chicago_ride_data).transform(chicago_ride_data).drop('payment_type').withColumnRenamed('payment_type_num', 'payment_type')
chicago_ride_data = sm
si = StringIndexer(inputCol = 'day_of_week', outputCol = 'day_of_week_num')
sm = si.fit(chicago_ride_data).transform(chicago_ride_data).drop('day_of_week').withColumnRenamed('day_of_week_num', 'day_of_week')
chicago_ride_data = sm
si = StringIndexer(inputCol = 'hour_of_start', outputCol = 'hour_of_start_num')
sm = si.fit(chicago_ride_data).transform(chicago_ride_data).drop('hour_of_start').withColumnRenamed('hour_of_start_num', 'hour_of_start')
chicago_ride_data = sm

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
time.time() - time1

In [ ]:
time1 = time.time()

### Fare

In [102]:
onehotencoder = OneHotEncoderEstimator(inputCols=categorical_features,
                                 outputCols=[i+'_cat'for i in categorical_features])
onehotencoder = onehotencoder.fit(chicago_ride_data)
all_columns = [i+'_cat'for i in categorical_features]
all_columns.append('fare')
encoded = onehotencoder.transform(chicago_ride_data).select(all_columns)

va_linear = VectorAssembler(inputCols=[i+'_cat'for i in categorical_features],outputCol='features')
train_df_linear = va_linear.transform(encoded)
train_df_linear = train_df_linear.select('features','fare')
train_df_linear = train_df_linear.withColumnRenamed('fare','label')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [103]:
training_linear, test_linear = train_df_linear.randomSplit([0.8, 0.2], 11)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [104]:
lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrModel = lr.fit(training_linear)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [105]:
pred_linear = lrModel.transform(test_linear)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [106]:
ev = RegressionEvaluator(predictionCol='prediction', labelCol='label', metricName='mae')
ev.evaluate(pred_linear)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4.455246047942442

### trip_miles

In [107]:
onehotencoder = OneHotEncoderEstimator(inputCols=categorical_features,
                                 outputCols=[i+'_cat'for i in categorical_features])
onehotencoder = onehotencoder.fit(chicago_ride_data)
all_columns = [i+'_cat'for i in categorical_features]
all_columns.append('trip_miles')
encoded = onehotencoder.transform(chicago_ride_data).select(all_columns)

va_linear = VectorAssembler(inputCols=[i+'_cat'for i in categorical_features],outputCol='features')
train_df_linear = va_linear.transform(encoded)
train_df_linear = train_df_linear.select('features','trip_miles')
train_df_linear = train_df_linear.withColumnRenamed('trip_miles','label')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [108]:
training_linear, test_linear = train_df_linear.randomSplit([0.8, 0.2], 11)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [109]:
lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrModel = lr.fit(training_linear)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [110]:
pred_linear = lrModel.transform(test_linear)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [111]:
ev = RegressionEvaluator(predictionCol='prediction', labelCol='label', metricName='mae')
ev.evaluate(pred_linear)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2.0678435229379635

### trip_seconds

In [112]:
onehotencoder = OneHotEncoderEstimator(inputCols=categorical_features,
                                 outputCols=[i+'_cat'for i in categorical_features])
onehotencoder = onehotencoder.fit(chicago_ride_data)
all_columns = [i+'_cat'for i in categorical_features]
all_columns.append('trip_seconds')
encoded = onehotencoder.transform(chicago_ride_data).select(all_columns)

va_linear = VectorAssembler(inputCols=[i+'_cat'for i in categorical_features],outputCol='features')
train_df_linear = va_linear.transform(encoded)
train_df_linear = train_df_linear.select('features','trip_seconds')
train_df_linear = train_df_linear.withColumnRenamed('trip_seconds','label')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [113]:
training_linear, test_linear = train_df_linear.randomSplit([0.8, 0.2], 11)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [114]:
lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrModel = lr.fit(training_linear)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [115]:
pred_linear = lrModel.transform(test_linear)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [116]:
ev = RegressionEvaluator(predictionCol='prediction', labelCol='label', metricName='mae')
ev.evaluate(pred_linear)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

336.2214393250627

In [117]:
time.time() - time1

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'float' object has no attribute 'time'
Traceback (most recent call last):
AttributeError: 'float' object has no attribute 'time'

